# from week 1

# Loading the vector database

In [1]:
import os
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings

In [10]:
try:
    PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))
except:
    print("This should not be running on a notebook.")
    PROJECT_ROOT = os.path.abspath('.')
    
DATA_PATH = os.path.join(PROJECT_ROOT, "database_s3", "queplan_insurance")
DB_PATH = os.path.join(PROJECT_ROOT, "database_chromadb", "db")

This should not be running on a notebook.


In [11]:
vectorstore = Chroma(
    persist_directory=DB_PATH,
    embedding_function=GPT4AllEmbeddings()
)

# First retriever

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
docs = retriever.aget_relevant_documents("Que contrato me recomiendas")

In [15]:
await docs

[Document(page_content='obligaciones del contrato, salvo aquellas que por su naturaleza deben ser cumplidos por el asegurado. Las', metadata={'page': 18, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320200071.pdf'}),
 Document(page_content='4° Notificar al Asegurador, tan pronto sea posible una vez tomado conocimiento, de la ocurrencia de\ncualquier hecho que pueda constituir o constituya un siniestro, y\n \n5° Acreditar la ocurrencia del siniestro denunciado, y declarar fielmente y sin reticencia, sus circunstancias y\nconsecuencias.\n \nSi el Tomador o Contratante del seguro y el Asegurado son personas distintas, corresponde al Tomador o\nContratante el cumplimiento de las obligaciones del contrato, salvo aquellas que por su naturaleza deben ser\ncumplidas por el Asegurado. Las obligaciones del Tomador o Contratante podrán ser cumplidas por el\nAsegurado.', metadata={'page': 12, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\

# First Retrieval Chain

In [17]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain_community.llms.openai import OpenAI

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    chain_type="stuff",
    retriever=retriever,
    llm=OpenAI(),
    return_source_documents=True,
    verbose=True
)

In [20]:
def process_llm_response(response):
    print(response['result'])
    print('\n\nSources:')
    [print(source.metadata['source']) for source in response['source_documents']]
    print("\n\n")

In [22]:
msg = 'Que contrato me recomiendas'
response = qa_chain.invoke(msg)



> Entering new RetrievalQA chain...

> Finished chain.


In [23]:
response

{'query': 'Que contrato me recomiendas',
 'result': ' No puedo recomendar un contrato en particular ya que soy un programa de ordenador y no tengo conocimiento de las necesidades o preferencias específicas de cada individuo. Es importante que consultes con un experto o investigues detenidamente antes de decidir qué contrato es el mejor para ti.',
 'source_documents': [Document(page_content='obligaciones del contrato, salvo aquellas que por su naturaleza deben ser cumplidos por el asegurado. Las', metadata={'page': 18, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320200071.pdf'}),
  Document(page_content='4° Notificar al Asegurador, tan pronto sea posible una vez tomado conocimiento, de la ocurrencia de\ncualquier hecho que pueda constituir o constituya un siniestro, y\n \n5° Acreditar la ocurrencia del siniestro denunciado, y declarar fielmente y sin reticencia, sus circunstancias y\nconsecuencias.\n \nSi el Tomador o Contrata

In [24]:
process_llm_response(response)

 No puedo recomendar un contrato en particular ya que soy un programa de ordenador y no tengo conocimiento de las necesidades o preferencias específicas de cada individuo. Es importante que consultes con un experto o investigues detenidamente antes de decidir qué contrato es el mejor para ti.


Sources:
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200071.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320180100.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320150503.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200214.pdf





In [25]:
msg = 'Necesito una receta de mayonesa'
response = qa_chain.ainvoke(msg)

In [26]:
result = await response
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Necesito una receta de mayonesa',
 'result': ' No conozco una receta de mayonesa.',
 'source_documents': [Document(page_content='que ella y sus coberturas se encuentren vigentes a esa fecha y que haya transcurrido el período de carencia\nseñalado en las Condiciones Particulares.', metadata={'page': 1, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320130223.pdf'}),
  Document(page_content='que ella y sus coberturas se encuentren vigentes a esa fecha y que haya transcurrido el período de carencia\nseñalado en las Condiciones Particulares.', metadata={'page': 16, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320190074.pdf'}),
  Document(page_content='cuyo reembolso se solicita, salvo que se indique otro plazo en las Condiciones Particulares de la póliza. Con\ntodo, en caso de no renovación de la póliza el plazo de aviso de siniestro no podrá ser superior a trein

# Inspeccion de los parametros del Retriever

In [27]:
qa_chain.retriever.search_type

'similarity'

In [ ]:
qa_chain.retriever.search_type = "semantic"

In [28]:
qa_chain.retriever.vectorstore

In [29]:
# System Prompt and Human Prompt:

print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Second Retrieval Chain

In [31]:
from langchain_openai import ChatOpenAI


turbo_llm = ChatOpenAI(
    temperature=0.1777,
    model_name='gpt-3.5-turbo'
)

qa_chain_turbo = RetrievalQA.from_chain_type(
    chain_type="stuff",
    retriever=retriever,
    llm=turbo_llm,
    return_source_documents=True,
    verbose=True
)

In [32]:
msg = 'Requiero recomendaciones para un ARTICULO: "cobertura"'
llm_response = qa_chain_turbo.invoke(msg)



> Entering new RetrievalQA chain...

> Finished chain.


In [33]:
process_llm_response(llm_response)

Para redactar un artículo sobre "cobertura" en el contexto de seguros, podrías considerar los siguientes puntos:

1. **Definición de Cobertura**: Explica qué se entiende por cobertura en el ámbito de los seguros. Puedes mencionar que se refiere a las garantías y protecciones que ofrece una póliza de seguro en caso de determinados eventos.

2. **Tipos de Cobertura**: Describe los diferentes tipos de cobertura que suelen ofrecer las pólizas de seguros, como cobertura médica, cobertura de responsabilidad civil, cobertura de daños materiales, entre otros.

3. **Importancia de la Cobertura**: Destaca la importancia de contar con una cobertura adecuada en un seguro, ya que brinda tranquilidad y protección financiera en situaciones imprevistas.

4. **Periodo de Cobertura**: Menciona que la cobertura de un seguro suele tener un periodo determinado, que puede variar según el tipo de póliza y las condiciones particulares del contrato.

5. **Exclusiones y Limitaciones**: Es importante mencionar q

In [36]:
# System Prompt
print(qa_chain_turbo.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [37]:
# Human Prompt
print(qa_chain_turbo.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}


# from week 2: 29/02/2024

In [1]:
import pickle, os

print(os.getcwd())

with open('pdf_dict.pkl', 'rb') as f:
    data = pickle.load(f)
    
query = "Articulos que mencionen siniestros"

c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend


In [9]:
data.keys()

dict_keys(['POL120190177.pdf', 'POL320130223.pdf', 'POL320150503.pdf', 'POL320180100.pdf', 'POL320190074.pdf', 'POL320200071.pdf', 'POL320200214.pdf', 'POL320210063.pdf', 'POL320210210.pdf'])

In [10]:
data["POL120190177.pdf"].keys()

dict_keys(['text', 'chunks', 'metadatas', 'retriever'])

In [11]:
type(data["POL120190177.pdf"]["chunks"])
len(data["POL120190177.pdf"]["chunks"])

110

In [12]:
type(data["POL120190177.pdf"]["text"])

str

In [16]:
type(data["POL120190177.pdf"]["retriever"])
"invoke"
"get_relevant_documents"
dir(data["POL120190177.pdf"]["retriever"])[-50:-24]

['config_specs',
 'configurable_alternatives',
 'configurable_fields',
 'construct',
 'copy',
 'dict',
 'docs',
 'from_documents',
 'from_orm',
 'from_texts',
 'get_graph',
 'get_input_schema',
 'get_lc_namespace',
 'get_name',
 'get_output_schema',
 'get_prompts',
 'get_relevant_documents',
 'input_schema',
 'invoke',
 'is_lc_serializable',
 'json',
 'k',
 'lc_attributes',
 'lc_id',
 'lc_secrets',
 'map']

In [3]:
retriever = data["POL120190177.pdf"]["retriever"]
type(retriever)

langchain_community.retrievers.bm25.BM25Retriever

In [6]:
rta = retriever.invoke("Articulos que mencionen siniestros")#get_relevant_documents(
rta

[Document(page_content='aseguradora sobre los riesgos que asume y ésta no tendrá obligación alguna respecto de los siniestros que\nocurran con posterioridad a esa fecha.\nARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA\nINDIVIDUAL\nSi durante la vigencia del seguro, un Asegurado Titular deseare incorporar a nuevos asegurados\ndependientes, deberá solicitarlo a la Compañía de Seguros. Para los efectos de incorporarse como\nasegurados, los nuevos dependientes deberán someterse a la evaluación y calificación de su estado de', metadata={'source': '92-lt'}),
 Document(page_content='(15) contado desde la notificación que, conforme al Artículo 21° de estas Condiciones Generales, dirija la\nCompañía Aseguradora al Asegurado y dará derecho al asegurador para exigir que se le pague la prima\ndevengada hasta la fecha de terminación y los gastos de formalización del contrato.\nProducida la terminación, la responsabilidad del asegurador por los siniestros posterior

In [22]:
print(rta[0].page_content)

aseguradora sobre los riesgos que asume y ésta no tendrá obligación alguna respecto de los siniestros que
ocurran con posterioridad a esa fecha.
ARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA
INDIVIDUAL
Si durante la vigencia del seguro, un Asegurado Titular deseare incorporar a nuevos asegurados
dependientes, deberá solicitarlo a la Compañía de Seguros. Para los efectos de incorporarse como
asegurados, los nuevos dependientes deberán someterse a la evaluación y calificación de su estado de


In [23]:
print(rta[1].page_content)

(15) contado desde la notificación que, conforme al Artículo 21° de estas Condiciones Generales, dirija la
Compañía Aseguradora al Asegurado y dará derecho al asegurador para exigir que se le pague la prima
devengada hasta la fecha de terminación y los gastos de formalización del contrato.
Producida la terminación, la responsabilidad del asegurador por los siniestros posteriores cesará de pleno
derecho, sin necesidad de declaración judicial alguna.
ARTÍCULO 9º: DENUNCIA DE SINIESTRO 
En caso que el Asegurado, a causa de un Evento, incurra en un gasto susceptible de ser reembolsado o


In [24]:
print(rta[2].page_content)

aportadas, bonificadas o reembolsadas al Asegurado por instituciones o entidades a las que se refiere el
párrafo anterior y que otorguen reintegros, aportes, bonificaciones o reembolsos médicos y prestaciones
económicas al Asegurado.
Queda expresamente establecido que las coberturas de esta Póliza no se duplicarán con los beneficios o
coberturas de cualquier otro seguro, sistema o institución que otorgue beneficios médicos, y al cual
pertenezca el asegurado, de manera que esta Póliza no cubrirá los gastos médicos que deben pagar o


In [25]:
"metadata"
dir(rta[2])[-25:]

['_init_private_attributes',
 '_iter',
 '_lc_kwargs',
 'construct',
 'copy',
 'dict',
 'from_orm',
 'get_lc_namespace',
 'is_lc_serializable',
 'json',
 'lc_attributes',
 'lc_id',
 'lc_secrets',
 'metadata',
 'page_content',
 'parse_file',
 'parse_obj',
 'parse_raw',
 'schema',
 'schema_json',
 'to_json',
 'to_json_not_implemented',
 'type',
 'update_forward_refs',
 'validate']

In [6]:
rta[2].metadata

{'source': '79-lt'}

In [7]:
"docs"
dir(retriever);

In [11]:
"docs"
retriever.docs
retriever.docs[0].metadata

{'source': '0-lt'}

In [13]:
# Obtaining the relevant document arround the recomendation
# Inspecting recomendation number 3 with index 2

recomendation = rta[2].metadata['source']
for i in range(len(retriever.docs)):
    if recomendation == retriever.docs[i].metadata['source']:
        relevant_docs = [
            retriever.docs[i-2].page_content,
            retriever.docs[i-1].page_content,
            retriever.docs[i].page_content,
            retriever.docs[i+1].page_content,
            retriever.docs[i+2].page_content
        ]
        break



In [14]:
print(relevant_docs[0])
print(relevant_docs[1])
print(relevant_docs[2])
print(relevant_docs[3])
print(relevant_docs[4])

ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES
El Asegurado, conforme al Artículo 9°, letra b), deberá declarar en su denuncia de siniestro sobre el sistema
previsional o de salud al que se encuentre afiliado o adherido, Isapres, Fonasa, cajas de previsión,
departamentos o servicios de bienestar, el Seguro Obligatorio de Accidentes Personales establecido en la
Ley Nº 18.490, las Cajas de Compensación de Asignación Familiar, otros Seguros de Salud contratados en
favor del Asegurado por otras instituciones, públicas o privadas, u otros seguros contratados en forma
individual por él, que contemplen algún aporte, indemnización, reembolso o bonificación, a favor del
Asegurado, efectuado o por efectuar, respecto de los gastos susceptibles de ser reembolsados o pagados al
Prestador.
Para los efectos de esta Póliza, el procedimiento de cálculo de los Gastos Reembolsables considerará
únicamente los Gastos Efectivamente Incurridos, esto es, la diferencia entre el monto total de los Gastos
Méd

### Proximos pasos:

* Crear un merger retriever (lotr) entre todos los documentos
* Aplicar un context compressor de redundancia
* Colocar el lotr en el pipeline
* (Opcional) Aplicar un context compressor de clustering
* (Opcional) crear un ensamble retriever y fijar weights a los documentos


### Proximos pasos 2:

* Crear prompts para el modelo
* Crear modelo

### Pendientes:

* Mejorar la metadata
* Hacer que el modelo responda en formato Streaming


### Consideraciones:

* Será provechoso que el parametro k del retriever dependa de la longitud del documento?


In [13]:
# VERSION SIN ORDENAMIENTO DE LOS DOCUMENTOS

from langchain.retrievers.merger_retriever import MergerRetriever
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_community.document_transformers.embeddings_redundant_filter import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors.base import DocumentCompressorPipeline
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever


# Create a basic query
query = "Articulos que mencionen siniestros"

# Isolate all the BM25 retrievers
retriever_list = [data[key]["retriever"] for key in data.keys()]

# Create a MergerRetriever
lotr = MergerRetriever(retrievers=retriever_list)

# Create embedding model for redundant compressor
embedding = GPT4AllEmbeddings()

# Create the redundant compressor filter
filter = EmbeddingsRedundantFilter(embeddings=embedding)

# Create a Document compressor pipeline 
pipeline = DocumentCompressorPipeline(transformers=[filter])

# Create a Contextual Compressor Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline,
    base_retriever=lotr
)

In [15]:
res2 = compression_retriever.invoke(query)
# Ok, que es State?
len(res2)

41

In [16]:
# VERSION CON ORDENAMIENTO DE LOS DOCUMENTOS

from langchain.retrievers.merger_retriever import MergerRetriever
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_community.document_transformers.embeddings_redundant_filter import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors.base import DocumentCompressorPipeline
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_transformers.long_context_reorder import LongContextReorder

# Create a basic query
query = "Articulos que mencionen siniestros"

# Isolate all the BM25 retrievers
retriever_list = [data[key]["retriever"] for key in data.keys()]

# Create a MergerRetriever
lotr = MergerRetriever(retrievers=retriever_list)

# Create embedding model for redundant compressor
embedding = GPT4AllEmbeddings()

# Create the redundant compressor filter
filter = EmbeddingsRedundantFilter(embeddings=embedding)

# Create a context reorder transformer
reorder = LongContextReorder()

# Create a Document compressor pipeline 
pipeline = DocumentCompressorPipeline(transformers=[filter, reorder])

# Create a Contextual Compressor Retriever
compression_reorder_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline,
    base_retriever=lotr
)

In [17]:
res3 = compression_reorder_retriever.invoke(query)
res3

[_DocumentWithState(page_content='coberturas adicionales que se contraten en conjunto, si las hubiere, se iniciará en la fecha indicada en las\nCondiciones Particulares de la Póliza, tendrá la duración señalada en las mismas Condiciones Particulares y\npodrá ser renovada por el lapso, condiciones y prima que autorice el Contratante.\n \nTerminada la vigencia del contrato, sea anticipada o no, cesará toda responsabilidad de la compañía\naseguradora sobre los riesgos que asume y ésta no tendrá obligación alguna respecto de los siniestros que\nocurran con posterioridad a esa fecha.', metadata={'source': '98-lt'}, state={'embedded_doc': [0.06240067258477211, 0.07807576656341553, -0.03649096563458443, -0.036765336990356445, -0.05944123491644859, 0.04963129013776779, 0.07582878321409225, -0.013223440386354923, 0.0014548691688105464, 0.028659701347351074, 0.12266930192708969, -0.007453553378582001, 0.03061573952436447, -0.008501463569700718, 0.04017416387796402, -0.04091279208660126, 0.004257

In [19]:
len(res3)

41

In [18]:
print(res3[0].page_content)

coberturas adicionales que se contraten en conjunto, si las hubiere, se iniciará en la fecha indicada en las
Condiciones Particulares de la Póliza, tendrá la duración señalada en las mismas Condiciones Particulares y
podrá ser renovada por el lapso, condiciones y prima que autorice el Contratante.
 
Terminada la vigencia del contrato, sea anticipada o no, cesará toda responsabilidad de la compañía
aseguradora sobre los riesgos que asume y ésta no tendrá obligación alguna respecto de los siniestros que
ocurran con posterioridad a esa fecha.


In [20]:
print(res3[1].page_content)

terminación y los gastos de formalización del contrato.Producida la terminación, la responsabilidad del asegurador por los siniestros posteriores cesará de pleno
derecho, sin necesidad de declaración judicial alguna.
 
ARTICULO 9°: BENEFICIARIOS
El beneficiario de esta póliza será el Asegurado Titular individualizado en las Condiciones Particulares, salvo
que se establezca uno diferente lo que deberá quedar expresamente establecido en las Condiciones
Particulares.
Siendo beneficiario el Asegurado Titular, en caso de fallecer éste antes del pago del beneficio asociado a


In [21]:
print(res3[2].page_content)

prestación progresiva de 2 (dos) años y para asegurados que hayan cumplido los 60 (sesenta) años de
edad.
 
 
44) Tratamientos para el crecimiento: Cualquier tratamiento relacionado con la hormona del crecimiento,
incluyendo tratamientos realizados por un estimulador de crecimiento óseo, excepto cuando esté
directamente relacionado al tratamiento de una condición cubierta bajo la póliza.
 
 45) Tratamientos que provengan de siniestros ocurridos en alguno de los países sancionados que aparecen


In [22]:
print(res3[3].page_content)

asegurado, según se especifique en las Condiciones Particulares de la póliza.
La falta de pago de la prima producirá la terminación del contrato a la expiración del plazo de quince días
contado desde la fecha de envío de la comunicación que, con ese objeto, dirija el asegurador al asegurado o
Contratante y dará derecho a aquél para exigir que se le pague la prima devengada hasta la fecha de
terminación y los gastos de formalización del contrato.
Producida la terminación, la responsabilidad del asegurador por los siniestros posteriores cesará de pleno


Conclusiones:


Al parecer se tiene un motor de búsqueda bastante versátil.
Cada PDF tiene un retriever asignado, el factor k es el mismo para todos los retrievers.
Excepto por el source 'lt' no existe más metadata que le sirva al developer para verificar la relevancia de los resultados.
Parece que hiciera falta un paso previo a la búsqueda de documentos relevantes.

- Metadata: Un proceso que identifique 'nombre', 'pagina', 'articulo' correspondiente a la recomendacion del retriever.
- La falta de prompts.

Por último falta crear una instancia de LLM, la chain y el agent. Y la interace UI para el user.

## Probando Main Retriever Serializado


In [1]:
import pickle

with open("main_retriever.pkl", "rb") as f:
    main_retriever = pickle.load(f)

In [2]:
main_retriever.invoke("Poliza de seguridad")

OSError: exception: access violation reading 0x0000017369A95170

## Probando el BM25 retriever

In [3]:
import pickle

with open("pdf_dict.pkl", "rb") as f:
    pdf_dict = pickle.load(f)

In [4]:
archivo_1_name = list(pdf_dict.keys())[0]

pdf_dict[archivo_1_name]["retriever"].invoke("Poliza de seguridad")

[Document(page_content='deportes extremos tales como parapente, benji, canopy, paracaidismo, alas delta u otros del mismo género y\nen general aquellas actividades que requieren el uso de protecciones y medidas especiales de seguridad\npara garantizar la integridad física de quien lo practica.\ns) Lesión a consecuencia de la práctica como deportista profesional de alto rendimiento según la Ley\n19.712., cuando no haya sido informado a la compañía y aceptado explícitamente por ésta al momento de', metadata={'source': '58-lt'}),
 Document(page_content='de cualquier tipo incluyendo las funciones de policía civil y gendarmería.\nv) Adquisición o arriendo de equipos, tales como: sillas de rueda, camas médicas, ventiladores mecánicos,\netc.\nw) Tratamientos efectuados a través de medios de medicina alternativa, tales como: acupuntura, iriólogos,\nreflexología, quiropraxia, y otros similares.\nx) Gastos de exámenes y procedimientos a través de nuevas tecnologías no reconocidas por el Sistema 

# Week 3: Creando base de datos vectorizada

In [20]:
from qdrant_client import QdrantClient
from langchain_community.vectorstores.qdrant import Qdrant
from ml_service.embeddings import Embeddings
from utils.config import QVDB_BASE_PATH

# GPT4ALL
client = QdrantClient(path=QVDB_BASE_PATH.format(embeddings=Embeddings().get_current(), filename="all_files"))
collection_name = "all_documents"
qdrant = Qdrant(client, collection_name, Embeddings().obtain_embeddings())


In [29]:
retriever = qdrant.as_retriever()

res = retriever.invoke("salud")
res

[Document(page_content='de la enfermedad o situación de salud correspondiente a las enfermedades graves cubiertas por este\ncontrato de seguro que pueda constituir o constituya un siniestro, de acuerdo a los procedimientos y a través\nde los medios que esta última disponga para dichos efectos, los que se indican en las Condiciones\nParticulares de la póliza.\nNo obstante lo anterior, la compañía aseguradora aceptará que el siniestro pueda ser notificado a la', metadata={'source': 'C:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320200071.pdf', 'page': 20, '_id': '2161dac833e247fe902068e90868ad68', '_collection_name': 'all_documents'}),
 Document(page_content='de la enfermedad o situación de salud correspondiente a las enfermedades graves cubiertas por este\ncontrato de seguro que pueda constituir o constituya un siniestro, de acuerdo a los procedimientos y a través\nde los medios que esta última disponga para dichos efectos, los que se indi

In [30]:
print(res[0].page_content)
len(res[0].page_content)

de la enfermedad o situación de salud correspondiente a las enfermedades graves cubiertas por este
contrato de seguro que pueda constituir o constituya un siniestro, de acuerdo a los procedimientos y a través
de los medios que esta última disponga para dichos efectos, los que se indican en las Condiciones
Particulares de la póliza.
No obstante lo anterior, la compañía aseguradora aceptará que el siniestro pueda ser notificado a la


434

In [31]:
# OPEN AI ada
client = QdrantClient(path=QVDB_BASE_PATH.format(embeddings=Embeddings().get_embeddings()[0], filename="all_files"))
collection_name = "all_documents"
qdrant = Qdrant(client, collection_name, Embeddings().obtain_embeddings(Embeddings().get_embeddings()[0]))

In [32]:
retriever = qdrant.as_retriever()

res = retriever.invoke("poliza de salud")
res

[Document(page_content='28. Póliza: El documento justificativo del seguro. Se encuentra compuesto fundamentalmente por la\npropuesta, la declaración personal de salud, las condiciones generales, condiciones particulares y las\ndeclaraciones, informaciones y antecedentes proporcionados por el asegurado a solicitud de la Compañía\nAseguradora. En ella se especifican las condiciones bajo las cuales rige la cobertura.\n29. Prestador de Salud Preferente o Prestador: Es aquel establecimiento hospitalario que se individualiza\ncomo preferente en las condiciones particulares de esta Póliza. El prestador es aquel que le cobra\ndirectamente al Asegurado o factura los Gastos Reembolsables correspondientes del servicio que debe\nasumir la Aseguradora.\n30. Propuesta de Seguros: La oferta escrita de contratar el seguro, formulada al asegurador por el\ncontratante, o por un tercero a su nombre.\n31. Prótesis: Es un dispositivo diseñado para reemplazar una parte faltante del cuerpo o para hacer que u

In [33]:
print(res[0].page_content)
len(res[0].page_content)

28. Póliza: El documento justificativo del seguro. Se encuentra compuesto fundamentalmente por la
propuesta, la declaración personal de salud, las condiciones generales, condiciones particulares y las
declaraciones, informaciones y antecedentes proporcionados por el asegurado a solicitud de la Compañía
Aseguradora. En ella se especifican las condiciones bajo las cuales rige la cobertura.
29. Prestador de Salud Preferente o Prestador: Es aquel establecimiento hospitalario que se individualiza
como preferente en las condiciones particulares de esta Póliza. El prestador es aquel que le cobra
directamente al Asegurado o factura los Gastos Reembolsables correspondientes del servicio que debe
asumir la Aseguradora.
30. Propuesta de Seguros: La oferta escrita de contratar el seguro, formulada al asegurador por el
contratante, o por un tercero a su nombre.
31. Prótesis: Es un dispositivo diseñado para reemplazar una parte faltante del cuerpo o para hacer que una


968

In [38]:
res[0].metadata

{'source': 'C:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL120190177.pdf',
 'page': 5,
 '_id': '1798034a9a184dc18cf8a197680d3b58',
 '_collection_name': 'all_documents'}